In [ ]:
import pickle
import torch
import torch.nn as nn
import numpy as np

import sys
sys.path.append("models")
from VGG_16 import VGG16
import os
import torchvision
import torchvision.transforms as transforms
from collections import Counter
import torchvision.datasets as datasets
import random
import torch.utils.data as Data
import torch.nn.functional as F
import argparse
from resnet import *
from model_mask_resnet import *
from model_mask_vgg import mask_VGG16
from AlexNet_SVHN_activation import AlexNet

In [ ]:
ori_model = AlexNet(num_classes=10)
model_path = "./trained_models/alexnet_lr0.0001_39.pkl"
checkpoint = torch.load(model_path)
ori_model.load_state_dict(checkpoint)

ori_model = ori_model.cuda()
ori_model.eval()

In [ ]:
transform_test = transforms.Compose([
            transforms.ToTensor(),
    ])

dataset = torchvision.datasets.SVHN(
        root = './data/SVHN',
        split="train",
        transform = transform_test,
        download = False)
data_loader = Data.DataLoader(dataset=dataset, batch_size=1, shuffle=False)

In [ ]:
feature_size = [64, 192, 384, 256, 256]

start_index = 0
mean_features_specialUnits = {}
mean_features_regularUnits = {}
num_special = {}
num_regular = {}

with torch.no_grad():
    sample_num = 0
    for step, (x, y) in enumerate(dataloader):    
        print("step:", step)
        batch_size = x.size(0)
        start_index += x.size(0)
        x = x.cuda()       
        target, feature_dict = model(x)      
        target = torch.max(target.data, 1)[1].data
      
        for k in range(5):
            units = paths[start_index][][k]
            conv_name = "feat_conv{}_relu".format(k+1)
            for i, unit in enumerate(units):
                if step == 0:
                    num_special[conv_name] += 1
                m = feature_dict[conv_name][:, unit, :, :]
                mean_features_specialUnits[conv_name] += m.abs().sum().item()

        for k in range(5):
            conv_name = "feat_conv{}_relu".format(k+1)        
            for unit in range(feature_dict[conv_name].size(1)):            
                if not unit in units_clean[k]:
                    if step == 0:
                        num_regular[conv_name] += 1
                    m = feature_dict[conv_name][:, unit, :, :]
                    mean_features_regularUnits[conv_name] += m.abs().sum().item()
                    
for k in mean_features_specialUnits.keys():
    print(mean_features_specialUnits[k] / num_special[k] / sample_num)
print("!!!")
for k in mean_features_regularUnits.keys():
    print(mean_features_regularUnits[k] / num_regular[k] / sample_num)
# print(mean_feature_abs)

print(num_regular, num_special)